In [1]:
from skimage.feature import hog
from skimage.feature import local_binary_pattern
import cv2 
import os
import numpy as np
import tqdm
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

In [2]:
train_pos = r"split\train_positif"
train_neg = r"split\train_negatif"
test_pos = r"split\test_positif"
test_neg = r"split\test_negatif"

In [3]:
def extract_feature_combined(live_path, spoof_path):
    # List to store extracted features of an image
    features = []
    
    # List to store class labels, 1 for live, 0 for spoof
    labels = []
    
    radius = 3
    # Number of neighbors to consider for LBP
    n_points = 8 * radius
    # Sampling type for LBP
    METHOD = 'uniform'
    
    path_array = [live_path, spoof_path]
    
    for path in path_array:
        # Storing all images in a folder in a list 'files'
        files = os.listdir(path)
        
        # Loop through the images in the folder
        for img in files:
            # Construct the full image path
            path_img = os.path.join(path, img)
            # Reading the image in grayscale using cv2
            img = cv2.imread(path_img, cv2.IMREAD_GRAYSCALE)
            
            # Resizing the image so all images are of same size
            resized_img = cv2.resize(img, (300, 300))
            
            # Extracting features using LBP
            lbp = local_binary_pattern(resized_img, n_points, radius, METHOD)
            lbp_features = lbp.flatten()
            
            # Extracting features using HOG
            hog_features, _ = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), visualize=True)
            
            # Concatenating LBP and HOG features
            combined_features = np.concatenate((lbp_features, hog_features))
            
            # Label 1 for live images, 0 for spoof images
            class_identifier = 1
            if 'neg' in path:
                class_identifier = 0
            
            # Appending extracted features to the list
            features.append(combined_features)
            
            # Adding corresponding class label to the list
            labels.append(class_identifier)
    
    return features, labels

In [5]:
# Training and testing datasets
com_x_trn,com_y_trn = extract_feature_combined(train_pos,train_neg)
com_x_tst,com_y_tst = extract_feature_combined(test_pos, test_neg)

# Create and fit the model
com_clf = svm.SVC()
com_clf.fit(com_x_trn,com_y_trn)

# Predict on the test features, print the results
com_y_pred = com_clf.predict(com_x_tst)
print("Combined Accuracy: "+str(accuracy_score(com_y_tst, com_y_pred)))
print('\n')
print(classification_report(com_y_tst, com_y_pred))

Combined Accuracy: 0.8387096774193549


              precision    recall  f1-score   support

           0       0.77      0.97      0.86        31
           1       0.96      0.71      0.81        31

    accuracy                           0.84        62
   macro avg       0.86      0.84      0.84        62
weighted avg       0.86      0.84      0.84        62

